In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import glob
import os
import importlib
from raster_utils import file_utils, band_indices
import datetime as dt
import matplotlib.pyplot as plt

#importlib.reload(band_indices)
#importlib.reload(file_utils)

In [2]:
#Files
data_dir = '/Users/miksch/Thesis_Files/Raw_Data/Landsat/EL_fmask/'
out_dir = '/Users/miksch/Thesis_Files/Processed/Landsat/EL_LS8/'
ls8_sr_files = glob.glob(os.path.join(data_dir,'LS8','*.tif'))
ls8_dn_files = glob.glob(os.path.join(data_dir,'LS8_DN','*.tif'))
nlcd_file = os.path.join(data_dir,'NLCD','NLCD2011.tif')

ls8_sr_bands = pd.read_csv('band_list_ls8_sr.csv',index_col=[0])['Name']
nlcd_bands = pd.read_csv('band_list_nlcd_2011.csv',index_col=[0])['Name']

In [3]:
#Landsat 8
#Master xarray dataarray

ls8_sr = file_utils.append_rasterio(files=ls8_sr_files,band_lookup=ls8_sr_bands)
nlcd = xr.open_rasterio(nlcd_file)
nlcd = file_utils.band_to_var(nlcd,nlcd_bands)
ls8_dn = file_utils.append_rasterio(ls8_sr_files,rewrite_bands=False).sel(band=slice(2,7))

#Merge together NLCD
ls8_master = xr.merge([ls8_sr,nlcd,ls8_dn])
ls8_master.attrs = ls8_sr.attrs
ls8_master['qa_mask'] = ls8_master['pixel_qa'].isin([322, 386, 834, 898, 1346])
ls8_master.to_netcdf(os.path.join(out_dir,'ls8_nomasks.nc'))

In [ ]:
ls8_master = ls8_master.where(ls8_master['qa_mask'],np.nan)
ls8_master = ls8_master.where(ls8_master['impervious'] < 75,np.nan)

In [4]:
#Calculate BCI and NDBI
bci_list = []
for t in ls8_master['time']:
    temp_bci = band_indices.BCI(ls8_master['band_values'].sel(time=t).values)
    bci_list.append(temp_bci)
    
ls8_master['BCI'] = (('time','y','x'),np.array(bci_list))
ls8_master['NDBI'] = band_indices.NDBI(ls8_master['B6'],ls8_master['B5'])

ls8_master = ls8_master.drop(['band','band_values'])

/Users/miksch/git/triangle-method/tri_method_jupyter/raster_utils/band_indices.py:86: RuntimeWarning: All-NaN slice encountered
  HVL_maxes = np.nanmax(tcap,axis=(1,2))[:,np.newaxis,np.newaxis]
/Users/miksch/git/triangle-method/tri_method_jupyter/raster_utils/band_indices.py:87: RuntimeWarning: All-NaN slice encountered
  HVL_mins = np.nanmin(tcap,axis=(1,2))[:,np.newaxis,np.newaxis]


In [5]:
#Output to .nc files
for t in ls8_master['time'].values:
    print(t)
    time_str = pd.to_datetime(t).strftime('%Y%m%d')
    ls8_master.sel(time=t).to_netcdf(os.path.join(out_dir,'v01',f'ls8_{time_str}_v01.nc'))

ls8_master.to_netcdf(os.path.join(out_dir,f'ls8_v01.nc'))

2016-05-16T00:00:00.000000000
2016-06-01T00:00:00.000000000
2016-06-17T00:00:00.000000000
2016-07-03T00:00:00.000000000
2016-07-19T00:00:00.000000000
2016-08-04T00:00:00.000000000
2016-08-20T00:00:00.000000000
2016-09-05T00:00:00.000000000
2016-09-21T00:00:00.000000000
2016-10-07T00:00:00.000000000
2016-10-23T00:00:00.000000000
2017-05-03T00:00:00.000000000
2017-06-04T00:00:00.000000000
2017-06-20T00:00:00.000000000
2017-07-06T00:00:00.000000000
2017-07-22T00:00:00.000000000
2017-08-07T00:00:00.000000000
2017-08-23T00:00:00.000000000
2017-09-08T00:00:00.000000000
2017-10-10T00:00:00.000000000
2017-10-26T00:00:00.000000000
2018-05-06T00:00:00.000000000
2018-05-22T00:00:00.000000000
2018-06-07T00:00:00.000000000
2018-06-23T00:00:00.000000000
2018-07-09T00:00:00.000000000
2018-07-25T00:00:00.000000000
2018-08-10T00:00:00.000000000
2018-08-26T00:00:00.000000000
2018-09-11T00:00:00.000000000
2018-09-27T00:00:00.000000000
2018-10-13T00:00:00.000000000
